# Client Profile
Facts about an individual


In [27]:
import ReportTimerange as rtr
import ReportMailer as rm
import GuestbookDbConnect as gdb

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days

In [28]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

In [29]:
import os
import pandas as pd

path = "/home/gbadmin/jupy-notebooks/Reports/inqueue"
dir_list = os.listdir(path)

if len(dir_list)>0:
  filename = "inqueue/{}".format(dir_list[0])
  print(filename)

  rep = pd.read_json(filename, lines=True)

  for _ , row in rep.iterrows():    
    _IDPERSON = row['idperson']
    _INTERVAL = 'PW'
 
  # delete the file
  os.remove(filename)  
else:
  _IDPERSON = 82
  _INTERVAL = 'PW'

print(_IDPERSON)

82


In [30]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [31]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import GuestbookDbConnect as gdb

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


# Basic Information

In [40]:
collection = {}
collection["name"] = "Basic Information"

collections = []

person_query = "select idperson, firstname, lastname, aliasname as alias, \
                'registeredSexOffender_id' as sexoffender,  veteran_id as verteran,  credits, \
                timelinestarttime as timelinestart,  iscurrentlysuspended as currentlysuspended\
                from guestbook_person \
                WHERE idperson={}".format(_IDPERSON)

#print(person_query)

person = pd.read_sql(person_query, conn)

collection["dataframe"] = person 
collection["colwidths"] = [30, 20]
collections.append(collection)

print(person)

print("index {}".format(person.index))
print("columns {}".format(person.columns))

print("start date {}".format(person['firstname']))

   idperson firstname lastname  alias               sexoffender  verteran  \
0        82      RECO     REID  DUMMY  registeredSexOffender_id         3   

   credits             timelinestart  currentlysuspended  
0       32 2018-05-28 00:00:01+00:00               False  
index RangeIndex(start=0, stop=1, step=1)
columns Index(['idperson', 'firstname', 'lastname', 'alias', 'sexoffender', 'verteran',
       'credits', 'timelinestart', 'currentlysuspended'],
      dtype='object')
start date 0    RECO
Name: firstname, dtype: object


In [25]:
trange = rtr.timerange(_INTERVAL)
print(trange)

[datetime.datetime(2022, 1, 23, 0, 0), datetime.datetime(2022, 1, 29, 23, 59, 59, 999999), 'YYYY-MM/DD DY']


# Create and mail spreadsheet

In [26]:
_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Reports for {}".format(toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])

_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

Writing dataframe to Excel file PW-2022-01-30_142754.xlsx
Writing Basic Information to spreadsheet


TypeError: Excel doesn't support timezones in datetimes. Set the tzinfo in the datetime/time object to None or use the 'remove_timezone' Workbook() option